# Testing neural network

In [1]:
import sys
import os
import matplotlib.pyplot as plt
sys.path.append("neural_network")
from trainer import Trainer
from nn_model import NNModel
from ann_model import ANNModel
from cnn_model import CNNModel
from t_cnn_model import TCNModel
from hyperparameter_manager import HyperparameterManager
from waveform_batch_manager import WaveformBatchManagerHDF5
import nn_transforms

import datetime

param_manager = HyperparameterManager()

## ANN model

In [2]:
parameters_ANN = param_manager.get_ann_parameters()
trainer_ann = Trainer(ANNModel, parameters_ANN, ["../data/raw/PAM_1_10.23", "../data/raw/PWM_1_22.27"], max_first_row_=1, max_stride_rows_=1, batch_size_=32, num_workers_=2)

In [3]:
trainer_ann.eval()

Finished evaluation in 3.04400444 seconds
Accuracy : 0.46875


In [4]:
trainer_ann.train(2)

[1,    20] loss: 20.067
[1,    40] loss: 19.698
[1,    60] loss: 17.996
[1,    80] loss: 16.828
[1,   100] loss: 15.697
[1,   120] loss: 13.993
[1,   140] loss: 13.466
[1,   160] loss: 13.056
[1,   180] loss: 11.782
[1,   200] loss: 10.827
[1,   220] loss: 9.700
[1,   240] loss: 9.569
[1,   260] loss: 8.818
[1,   280] loss: 7.172
[1,   300] loss: 7.422
[1,   320] loss: 5.879
[1,   340] loss: 5.813
[1,   360] loss: 5.638
[1,   380] loss: 5.374
[1,   400] loss: 4.237
[1,   420] loss: 4.335
[1,   440] loss: 4.923
[1,   460] loss: 3.653
[1,   480] loss: 3.108
[1,   500] loss: 3.502
[1,   520] loss: 3.458
[1,   540] loss: 3.470
[1,   560] loss: 3.070
Epoch 1 completed in 27.79 seconds
[2,    20] loss: 2.948
[2,    40] loss: 2.548
[2,    60] loss: 2.501
[2,    80] loss: 2.895
[2,   100] loss: 2.429
[2,   120] loss: 3.147
[2,   140] loss: 2.228
[2,   160] loss: 2.343
[2,   180] loss: 2.672
[2,   200] loss: 2.000
[2,   220] loss: 1.659
[2,   240] loss: 2.255
[2,   260] loss: 3.079
[2,   280] l

In [6]:
trainer_ann.eval()

Finished evaluation in 2.80147004 seconds
Accuracy : 1.0


## CNN model

In [2]:
parameters_CNN = param_manager.get_cnn_parameters_1d(nb_filter_cnn_A=12,nb_layer_A=3, nb_filter_cnn_B=16, nb_layer_B=5, nb_filter_cnn_C=16, nb_layer_C = 7, learning_rate=0.0005)
transform = nn_transforms.get_transform_to_2d()

train_set = WaveformBatchManagerHDF5('../data/raw/DatasetOnline/Modulation_4ASK_BPSK_QPSK.hdf5', ['4ASK','BPSK','QPSK'], 128, transform=transform)

trainer_cnn = Trainer(CNNModel, parameters_CNN, train_set=train_set, batch_size_=32, num_workers_=2, transform_ = transform)

In [3]:
trainer_cnn.eval()

13497
Finished evaluation in 8.27101684 seconds
Accuracy : 0.25


In [ ]:
trainer_cnn.train(12)

[1,  1000] loss: 0.982
[1,  2000] loss: 0.710
[1,  3000] loss: 0.609
[1,  4000] loss: 0.555
Epoch 1 completed in 157.15 seconds
Loss=0.706646030965931; Accuracy=64.31757897275139
[2,  1000] loss: 0.507
[2,  2000] loss: 0.474
[2,  3000] loss: 0.457
[2,  4000] loss: 0.438
Epoch 2 completed in 182.74 seconds
Loss=0.4677580410359939; Accuracy=78.71277429467085
[3,  1000] loss: 0.421
[3,  2000] loss: 0.406


## Eval Model for each SNR

In [35]:
snrs= range(-2, 22, 2)
accuracy_arr = []
for snr in snrs:
    total_accuracy = 0
    nb_it = 10
    for i in range(nb_it):
        total_accuracy += trainer_cnn.eval_by_snr(snr)
    average_accuracy = total_accuracy/nb_it
    accuracy_arr.append(average_accuracy)

print(accuracy_arr)

Finished evaluation in 1.45379257 seconds
Accuracy : 0.96875


In [ ]:
plt.plot(snrs, accuracy_arr, color="blue")

## Save Model

In [6]:
ABS_PATH = '~/School/PFE/PFE-A2023-Classification-Modulation/data/nn/CNN'
dir_path = os.path.expanduser(ABS_PATH)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'{timestamp}.pt'
full_path = os.path.join(dir_path, file_name)
print(full_path)

trainer_cnn.save_model(file_name, dir_path)

/home/will/School/PFE/PFE-A2023-Classification-Modulation/data/nn/CNN/20231201_105032.pt


## TCN Model

In [6]:
parameters_TCNN = param_manager.get_t_cnn_parameters(in_channels=1, num_layers=5, num_channels_each_layer=4)
transform = nn_transforms.get_transform_t_cnn()

trainer_tcnn = Trainer(TCNModel, parameters_TCNN, ["../data/raw/PAM_1_10.23", "../data/raw/PWM_1_22.27"], max_first_row_=5000, max_stride_rows_=10, batch_size_=32, num_workers_=2, transform_=transform)

In [7]:
trainer_tcnn.eval()

Finished evaluation in 3.28 seconds
Accuracy : 0.0


In [8]:
trainer_tcnn.train(2)

[1,    20] loss: 49.959
[1,    40] loss: 47.453
[1,    60] loss: 46.015
[1,    80] loss: 44.789
[1,   100] loss: 42.431
[1,   120] loss: 41.169
[1,   140] loss: 39.887
[1,   160] loss: 37.247
[1,   180] loss: 36.258
[1,   200] loss: 33.625
[1,   220] loss: 32.233
[1,   240] loss: 30.626
[1,   260] loss: 27.519
[1,   280] loss: 24.537
[1,   300] loss: 21.634
[1,   320] loss: 19.781
[1,   340] loss: 17.003
[1,   360] loss: 16.618
[1,   380] loss: 12.732
[1,   400] loss: 12.558
[1,   420] loss: 10.089
[1,   440] loss: 9.778
[1,   460] loss: 8.802
[1,   480] loss: 8.087
[1,   500] loss: 6.990
[1,   520] loss: 6.853
[1,   540] loss: 5.569
[1,   560] loss: 5.815
Epoch 1 completed in 27.83 seconds
[2,    20] loss: 4.332
[2,    40] loss: 3.971
[2,    60] loss: 3.832
[2,    80] loss: 3.715
[2,   100] loss: 3.583
[2,   120] loss: 3.706
[2,   140] loss: 2.992
[2,   160] loss: 2.523
[2,   180] loss: 2.681
[2,   200] loss: 2.713
[2,   220] loss: 2.350
[2,   240] loss: 2.293
[2,   260] loss: 2.116
[

In [9]:
trainer_tcnn.eval()

Finished evaluation in 4.95 seconds
Accuracy : 1.0
